In [1]:
import pandas as pd
import numpy as np
import pandas_profiling
import matplotlib.pyplot as plt
get_ipython().run_line_magic('matplotlib', 'inline')
import warnings # current version of seaborn generates a bunch of warnings that we'll ignore
warnings.filterwarnings("ignore")
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score

from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RandomizedSearchCV
from datetime import datetime

#this code below is to make %%time magic work
import glob
import os
from __future__ import print_function
#this code above is to make %%time magic work

In [2]:
#import the data
directory = 'C:/Users/N1110/Desktop/7331_Project/data/'
df = pd.read_csv(directory + 'Diabetes_Imputed_Cleaned.csv')
df_imputed = df
df_imputed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101766 entries, 0 to 101765
Columns: 245 entries, meds_increased to readmitted_tf
dtypes: int64(245)
memory usage: 190.2 MB


In [3]:
from sklearn.model_selection import ShuffleSplit

# we want to predict the X and y data as follows:
if 'readmitted_tf' in df_imputed:
    y = df_imputed['readmitted_tf'].values # get the labels we want
    del df_imputed['readmitted_tf'] # get rid of the class label
    X = df_imputed.values # use everything else to predict!

    ## X and y are now numpy matrices, by calling 'values' on the pandas data frames we
    #    have converted them into simple matrices to use with scikit learn
    
    
# to use the cross validation object in scikit learn, we need to grab an instance
#    of the object and set it up. This object will be able to split our data into 
#    training and testing splits
num_cv_iterations = 3
num_instances = len(y)
cv_object = ShuffleSplit(n_splits=num_cv_iterations,
                         test_size  = 0.2)
                         
print(cv_object)

ShuffleSplit(n_splits=3, random_state=None, test_size=0.2, train_size=None)


In [4]:
## Training and Testing Split
# okay, so run through the cross validation loop and set the training and testing variable for one single iteration
for train_indices, test_indices in cv_object.split(X,y): 
    # I will create new variables here so that it is more obvious what 
    # the code is doing (you can compact this syntax and avoid duplicating memory,
    # but it makes this code less readable)
    X_train = X[train_indices]
    y_train = y[train_indices]
    
    X_test = X[test_indices]
    y_test = y[test_indices]
    
        # we want to normalize the features based upon the mean and standard deviation of each column. 
# However, we do not want to accidentally use the testing data to find out the mean and std (this would be snooping)
# to Make things easier, let's start by just using whatever was last stored in the variables:
##    X_train , y_train , X_test, y_test (they were set in a for loop above)
from sklearn.preprocessing import StandardScaler
scl_obj = StandardScaler()

scl_obj.fit(X_train)
X_test_scaled = scl_obj.transform(X_test)

X_train_scaled = scl_obj.transform(X_train) # apply to training
X_test_scaled = scl_obj.transform(X_test) 

C:\Users\N1110\Anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
C:\Users\N1110\Anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
C:\Users\N1110\Anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
C:\Users\N1110\Anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


## How XGBoost work
#### boosting
A ML technique that iteratively combines a set of simple and not very accurate classifiers (referred to as "weak" classifiers) into a classifier with high accuracy (a "strong" classifier) by upweighting the examples that the model is currently misclassifying [3].

As opposed to the bagging where trees are built parallelly, in boosting, the trees are built sequentially such that each subsequent tree aims to reduce the errors of the previous tree. Each tree learns from its predecessors and updates the residual errors. Hence, the tree that grows next in the sequence will learn from an updated version of the residuals [2].  
1.	Fit a model to the data, F1(X)=Y
2.	Fit a model to the residuals, h1(X) = Y - F1(X)
3.	Create a new model, F2(X) = F1(X) + h1(X) [Note: F2 is boosted version of F1]
and goes on… Fm(X) = F(m-1)(X) + h(m-1)(X)

#### gradient descent
A technique to minimize loss by computing the gradients of loss with respect to the model's parameters, conditioned on training data. Informally, gradient descent iteratively adjusts parameters, gradually finding the best combination of weights and bias to minimize loss [3].

#### Gradient Boosting
Gradient Boosting employs the gradient descent algorithm to minimize errors in sequential models. The major inefficiency in Gradient Boosting is that it creates one decision tree at a time.
To overcome this, Tianqi Chen and Carlos Guestrin built A Scalable Tree Boosting System — XGBoost can be thought of as Gradient Boosting on steroids. It features parallelized tree building, cache-aware access, sparsity awareness, regularization, and
weighted quantile sketch as some of its systems optimization and algorithmic enhancements [8].

#### What is the difference between gradient boosting and adaboost?
They differ on how they create the weak learners during the iterative process.
At each iteration, adaptive boosting changes the sample distribution by modifying the weights attached to each of the instances. 
Gradient boosting doesn’t modify the sample distribution. Instead of training on a newly sample distribution, the weak learner trains on the remaining errors (so-called pseudo-residuals) of the strong learner [6].

#### learning rate

A scalar used to train a model via gradient descent. During each iteration, the gradient descent algorithm multiplies the learning rate by the gradient. The resulting product is called the gradient step.
Learning rate is a key hyperparameter [3].


#### What is XGBoost?
XGBoost is short for eXtreme gradient boosting. It is a library designed and optimized for boosted tree algorithms. XGBoost is a state-of-the-art a scalable end-to-end tree boosting system, and also have a novel sparsity-aware algorithm for sparse data [1]. It is a proven model in data science competition and hackathons for its accuracy, speed and scale [2].  For classification problems, XGBoost yield the similar or better accuracy with sk-learn but 10 x faster runtime [1].

In XGBoost, we fit a model on the gradient of loss generated from the previous step. In XGBoost, we just modified our gradient boosting algorithm so that it works with any differentiable loss function [2].  

#### How should I interpret XGBoost trees?
You can interpret xgboost model by interpreting individual trees [7].


## Advantages of XGBoost
#### Why does XGBoost perform better than SVM? 
In our project, we find SVM runs significantly longer than RF and XGBoost models, about 20 mins vs 2 mins vs 1 min. Also, the accuracy we got for these three models are:  XGBoost has around 0.75 auc while RF and SVM have similar auc of around 0.67 (see the comparison plot). RF actually has slightly higher auc than SVM.
XGBoost is an ensamble method it uses many trees to take a decision so it gains power by repeating itself. SVM is a linear separator, when data is not linearly separable SVM needs a Kernel to project the data into a space where it can separate it, there lies its greatest strength and weakness, by being able to project data into a high dimensional space SVM can find a linear separation for almost any data but at the same time it needs to use a Kernel and we can argue that there's not a perfect kernel for every dataset [4].

#### What makes xgboost run much faster than many other implementations of gradient boosting?
XGBoost is designed for scale in as its initial goal. Memory optimization, Cache-line optimization, The improvement in terms of model itself, distributed version; push the limit of the tool to fully utilize the resources, so you can run more examples on single node; There is an external memory version, that allows you to store data in disk, without loading all of them in memory [5].

Refs:
1.	Chen et al 2016. XGBoost: A Scalable Tree Boosting System https://www.kdd.org/kdd2016/papers/files/rfp0697-chenAemb.pdf

2.	https://medium.com/@pushkarmandot/how-exactly-xgboost-works-a320d9b8aeef

3.	https://developers.google.com/machine-learning/glossary
4.	https://www.quora.com/  Why does XGBoost perform better than SVM?
5.	https://www.quora.com/  What makes xgboost run much faster than many other implementations of gradient boosting?
6.	https://www.quora.com/  What is the difference between gradient boosting and adaboost?
7.	https://www.quora.com/ How should I interpret XGBoost trees?
8.  https://media.licdn.com/dms/document/C4D1FAQF_5vlXos139Q/feedshare-document-pdf-analyzed/0?e=1561834800&v=beta&t=GuChwPbOqD4OZz3vKAvfF81lRmtycXS6dooNNHeafTw From Zero to Hero in XGBoost Tuning

XGBoost Python Package
https://xgboost.readthedocs.io/en/latest/python/ 
https://github.com/dmlc/xgboost/tree/master/python-package



In [6]:
# XGBoost baseline model

### XGBoost

import xgboost as xgb


#XGBOOST parameters 1
MAX_ROUNDS = 1000
EARLY_STOP = 50
OPT_ROUNDS = 1000
VERBOSE_EVAL = 50
RANDOM_STATE = 2000

#XGBOOST transform data into DMatrix format for modeling
dtrain = xgb.DMatrix(X_train_scaled, y_train)
dvalid = xgb.DMatrix(X_test_scaled, y_test)
type(dtrain)


# XGBoost Parameters 2
params = {}
params['objective'] = 'binary:logistic'
#params['objective'] = 'multi:softmax'
#params['objective'] = 'reg:linear'
params['eta'] = 0.039
params['silent'] = True
params['max_depth'] = 2
params['subsample'] = 0.8
params['colsample_bytree'] = 0.9
params['eval_metric'] = 'auc'
params['random_state'] = RANDOM_STATE

watchlist = [(dtrain, 'train'), (dvalid, 'valid')]



[0]	train-auc:0.630101	valid-auc:0.626937
Multiple eval metrics have been passed: 'valid-auc' will be used for early stopping.

Will train until valid-auc hasn't improved in 50 rounds.
[50]	train-auc:0.71629	valid-auc:0.714845
[100]	train-auc:0.724855	valid-auc:0.722444
[150]	train-auc:0.730268	valid-auc:0.727276
[200]	train-auc:0.73447	valid-auc:0.731385
[250]	train-auc:0.737574	valid-auc:0.734527
[300]	train-auc:0.74058	valid-auc:0.737548
[350]	train-auc:0.742633	valid-auc:0.739162
[400]	train-auc:0.744783	valid-auc:0.741004
[450]	train-auc:0.746347	valid-auc:0.742047
[500]	train-auc:0.747737	valid-auc:0.743198
[550]	train-auc:0.749098	valid-auc:0.74403
[600]	train-auc:0.750151	valid-auc:0.744622
[650]	train-auc:0.751088	valid-auc:0.745281
[700]	train-auc:0.752051	valid-auc:0.745989
[750]	train-auc:0.75279	valid-auc:0.746322
[800]	train-auc:0.753532	valid-auc:0.746853
[850]	train-auc:0.75442	valid-auc:0.747438
[900]	train-auc:0.755043	valid-auc:0.747819
[950]	train-auc:0.755691	valid

In [8]:
%%time
xgb_clf = xgb.train(params,
                   dtrain,
                   MAX_ROUNDS,
                   watchlist,
                   early_stopping_rounds = EARLY_STOP,
                   maximize = True,
                   verbose_eval = VERBOSE_EVAL)


#Wall time: 3min 48s
#[999]	train-auc:0.756217	valid-auc:0.748447

[0]	train-auc:0.630101	valid-auc:0.626937
Multiple eval metrics have been passed: 'valid-auc' will be used for early stopping.

Will train until valid-auc hasn't improved in 50 rounds.
[50]	train-auc:0.71629	valid-auc:0.714845
[100]	train-auc:0.724855	valid-auc:0.722444
[150]	train-auc:0.730268	valid-auc:0.727276
[200]	train-auc:0.73447	valid-auc:0.731385
[250]	train-auc:0.737574	valid-auc:0.734527
[300]	train-auc:0.74058	valid-auc:0.737548
[350]	train-auc:0.742633	valid-auc:0.739162
[400]	train-auc:0.744783	valid-auc:0.741004
[450]	train-auc:0.746347	valid-auc:0.742047
[500]	train-auc:0.747737	valid-auc:0.743198
[550]	train-auc:0.749098	valid-auc:0.74403
[600]	train-auc:0.750151	valid-auc:0.744622
[650]	train-auc:0.751088	valid-auc:0.745281
[700]	train-auc:0.752051	valid-auc:0.745989
[750]	train-auc:0.75279	valid-auc:0.746322
[800]	train-auc:0.753532	valid-auc:0.746853
[850]	train-auc:0.75442	valid-auc:0.747438
[900]	train-auc:0.755043	valid-auc:0.747819
[950]	train-auc:0.755691	valid

In [7]:
preds = xgb_clf.predict(dvalid)
print('XGBoost - roc_auc_score: ', roc_auc_score(y_test, preds))

#XGBoost - roc_auc_score:  0.7484469077761282

XGBoost - roc_auc_score:  0.7484469077761282


## Hyperparameter Tuning - XGBoost

https://www.kaggle.com/tilii7/hyperparameter-grid-search-with-xgboost

We used both randomized search and grid search for RF models for comparsion purpose. To save time, we only run randomized search for XGBoost.

In addition, due to the limited computation power on my personal computer, I only run on 3-fold validation and limited parameter sampling for a reasonable runtime like 0.5-1 hr. Otherwise, it will take very long time like 5-10 hrs to run. Therefore, this search does not produce a great score (actually worse than baseline model).

Subsampling of 20% of our data also serves the purpose of save runtime. Plus, we found the results is similar with full dataset run.

In [32]:
#for the sake of time, subsample 20% of our data 

subsam_df_imputed=df_imputed.sample(frac=0.2)
subsam_df_imputed.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4071 entries, 19298 to 74072
Columns: 244 entries, meds_increased to age
dtypes: int64(244)
memory usage: 7.6 MB


In [33]:
df_imputed=subsam_df_imputed
df_imputed.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4071 entries, 19298 to 74072
Columns: 244 entries, meds_increased to age
dtypes: int64(244)
memory usage: 7.6 MB


In [34]:
from sklearn.model_selection import ShuffleSplit

# we want to predict the X and y data as follows:
if 'readmitted_tf' in df_imputed:
    y = df_imputed['readmitted_tf'].values # get the labels we want
    del df_imputed['readmitted_tf'] # get rid of the class label
    X = df_imputed.values # use everything else to predict!

    ## X and y are now numpy matrices, by calling 'values' on the pandas data frames we
    #    have converted them into simple matrices to use with scikit learn
    
    
# to use the cross validation object in scikit learn, we need to grab an instance
#    of the object and set it up. This object will be able to split our data into 
#    training and testing splits
num_cv_iterations = 3
num_instances = len(y)
cv_object = ShuffleSplit(n_splits=num_cv_iterations,
                         test_size  = 0.2)
                         
print(cv_object)

ShuffleSplit(n_splits=3, random_state=None, test_size=0.2, train_size=None)


In [35]:
## Training and Testing Split
# okay, so run through the cross validation loop and set the training and testing variable for one single iteration
for train_indices, test_indices in cv_object.split(X,y): 
    # I will create new variables here so that it is more obvious what 
    # the code is doing (you can compact this syntax and avoid duplicating memory,
    # but it makes this code less readable)
    X_train = X[train_indices]
    y_train = y[train_indices]
    
    X_test = X[test_indices]
    y_test = y[test_indices]

    # we want to normalize the features based upon the mean and standard deviation of each column. 
# However, we do not want to accidentally use the testing data to find out the mean and std (this would be snooping)
# to Make things easier, let's start by just using whatever was last stored in the variables:
##    X_train , y_train , X_test, y_test (they were set in a for loop above)
from sklearn.preprocessing import StandardScaler
scl_obj = StandardScaler()

scl_obj.fit(X_train)
X_test_scaled = scl_obj.transform(X_test)

X_train_scaled = scl_obj.transform(X_train) # apply to training
X_test_scaled = scl_obj.transform(X_test) 

C:\Users\N1110\Anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
C:\Users\N1110\Anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
C:\Users\N1110\Anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
C:\Users\N1110\Anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


## RandomizedSearchCV

In [19]:
def timer(start_time=None):
    if not start_time:
        start_time = datetime.now()
        return start_time
    elif start_time:
        thour, temp_sec = divmod((datetime.now() - start_time).total_seconds(), 3600)
        tmin, tsec = divmod(temp_sec, 60)
        print('\n Time taken: %i hours %i minutes and %s seconds.' % (thour, tmin, round(tsec, 2)))

In [42]:
# A parameter grid for XGBoost
#at least include the parameters already included in the baseline model for chance to find one better than base model.
params = {
        'min_child_weight': [1, 5],
        'gamma': [0.5, 1, 2],
        'subsample': [0.6, 0.8],
        'colsample_bytree': [0.6, 0.9],
        'max_depth': [2, 5]
        }

### watch out for total number of created models
difference combination of the features. Altogether, there are 2*2*2*2*2=32 settings.

In [43]:
from xgboost import XGBClassifier

xgb = XGBClassifier(learning_rate=0.02, n_estimators=600, objective='binary:logistic',
                    silent=True, nthread=1)

In [20]:
# seems XGBClassifier does not require to transform into DMatrix

### Grid search in a parallized fashion

In [44]:
# estimate runtime 40 mins
# actual runtime 30 mins
## grid search in a parallized fashion

from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RandomizedSearchCV
from datetime import datetime

folds = 3
param_comb = 3

skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 1001)

random_search = RandomizedSearchCV(xgb, param_distributions=params, n_iter=param_comb, scoring='roc_auc', n_jobs=4, cv=skf.split(X_train_scaled,y_train), verbose=50, random_state=2000)

# Here we go
start_time = timer(None) # timing starts from this point for "start_time" variable
random_search.fit(X_train_scaled, y_train)
timer(start_time) # timing ends here for "start_time" variable

Fitting 3 folds for each of 3 candidates, totalling 9 fits
[Parallel(n_jobs=4)]: Done   1 tasks      | elapsed:  5.7min
[Parallel(n_jobs=4)]: Done   2 tasks      | elapsed: 10.6min
[Parallel(n_jobs=4)]: Done   3 out of   9 | elapsed: 10.6min remaining: 21.2min
[Parallel(n_jobs=4)]: Done   4 out of   9 | elapsed: 10.6min remaining: 13.3min
[Parallel(n_jobs=4)]: Done   5 out of   9 | elapsed: 11.8min remaining:  9.4min
[Parallel(n_jobs=4)]: Done   6 out of   9 | elapsed: 17.2min remaining:  8.6min
[Parallel(n_jobs=4)]: Done   7 out of   9 | elapsed: 17.2min remaining:  4.9min
[Parallel(n_jobs=4)]: Done   9 out of   9 | elapsed: 17.9min remaining:    0.0s
[Parallel(n_jobs=4)]: Done   9 out of   9 | elapsed: 17.9min finished

 Time taken: 0 hours 24 minutes and 52.36 seconds.


In [28]:
print('\n Best estimator:')
print(random_search.best_estimator_)
print('\n Best normalized gini score for %d-fold search with %d parameter combinations:' % (folds, param_comb))
print(random_search.best_score_ * 2 - 1)
print('\n Best hyperparameters:')
print(random_search.best_params_)


 Best estimator:
XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.6, gamma=2, learning_rate=0.02, max_delta_step=0,
       max_depth=5, min_child_weight=1, missing=None, n_estimators=600,
       n_jobs=1, nthread=1, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=0.6)

 Best normalized gini score for 3-fold search with 3 parameter combinations:
0.5110818135734807

 Best hyperparameters:
{'subsample': 0.6, 'min_child_weight': 1, 'max_depth': 5, 'gamma': 2, 'colsample_bytree': 0.6}


### Results - Best hyperparameters
Best estimator:
XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.6, gamma=2, learning_rate=0.02, max_delta_step=0,
       max_depth=5, min_child_weight=1, missing=None, n_estimators=600,
       n_jobs=1, nthread=1, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=0.6)

 Best normalized gini score for 3-fold search with 3 parameter combinations:
0.5110818135734807

 Best hyperparameters:
{'subsample': 0.6, 'min_child_weight': 1, 'max_depth': 5, 'gamma': 2, 'colsample_bytree': 0.6}

In [37]:
#run for 
#evaulate the performance of the model best parameters
# timer the training time
start_time = timer(None)

xgb_best = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.6, gamma=2, learning_rate=0.02, max_delta_step=0,
       max_depth=5, min_child_weight=1, missing=None, n_estimators=600,
       n_jobs=1, nthread=1, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=0.6)

xgb_best.fit(X_train_scaled, y_train)

timer(start_time)



 Time taken: 0 hours 8 minutes and 24.05 seconds.


In [38]:
# timer the testing time
start_time = timer(None)
preds = xgb_best.predict(X_test_scaled)
timer(start_time)


 Time taken: 0 hours 0 minutes and 0.91 seconds.


C:\Users\N1110\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [39]:
print('XGB - roc_auc_score: ', roc_auc_score(y_test, preds)) 

XGB - roc_auc_score:  0.677932172132421


In [41]:
# The reason I want to keep both baseline model and refined model is to show the effect of Hyperparameter Tuning. 
# To see increase the AUC by what percentage.

XGBrandom_accuracy = 0.677932172132421
XGBbase_accuracy =0.748447

print('Improvement of {:0.2f}%.'.format( 100 * (XGBrandom_accuracy - XGBbase_accuracy) / XGBbase_accuracy))

Improvement of -9.42%.


### Not surprisingly, this search does not produce a great score because of 3-fold validation and limited parameter sampling. The randomized grid search is only for demo purpose. We should provide a larger list of parameters in practice.